In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/GovHackTrafficDeakin/gov hack nat

/content/drive/MyDrive/Colab Notebooks/GovHackTrafficDeakin/gov hack nat


# *Turn geojson files into csv
*: optional

In [ ]:
import geopandas as gpd

def read_geojson_file(file_path):
    return gpd.read_file(file_path)

def geojson_to_csv(geojson_data, csv_file_path):
    # Extract the necessary attributes and location
    geojson_data['latitude'] = geojson_data.geometry.y
    geojson_data['longitude'] = geojson_data.geometry.x
    # Drop the geometry column as it's no longer needed
    geojson_data = geojson_data.drop(columns='geometry')
    # Save to CSV
    geojson_data.to_csv(csv_file_path, index=False)

file_path = "Traffic_Lights.geojson"
csv_file_path = "Traffic_Lights_csv.csv"
geojson_data = read_geojson_file(file_path)
geojson_to_csv(geojson_data, csv_file_path)

# NODE.csv (Count num of accidents cases in a location)

In [ ]:
# Function to round LATITUDE and LONGITUDE to n decimal places, same as other dfs
def round_lat_lon(df, places=5):
    df['LATITUDE'] = df['LATITUDE'].round(places)
    df['LONGITUDE'] = df['LONGITUDE'].round(places)


In [ ]:
import pandas as pd
node_df = pd.read_csv('NODE.csv')
round_lat_lon(node_df, 4)

node_df.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,LGA NAME ALL,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH
0,T20120000009,249102,N,2563628.962,2362700.434,BAW BAW,BAW BAW,RURAL_VICTORIA,-38.2350,145.7267,3981
1,T20120000012,41780,N,2514194.683,2399718.586,MONASH,MONASH,MELB_URBAN,-37.9035,145.1614,3170
2,T20120000013,69811,I,2508689.410,2396130.003,KINGSTON,KINGSTON,MELB_URBAN,-37.9359,145.0988,3169
3,T20120000018,22636,I,2227360.371,2804744.001,MILDURA,MILDURA,RURAL_VICTORIA,-34.2159,142.0439,3505
4,T20120000021,248597,N,2480590.789,2347977.747,MORNINGTON PENINSULA,MORNINGTON PENINSULA,MELB_URBAN,-38.3696,144.7779,3942


# *check for duplicates in LATITUDE and LONGITUDE

In [ ]:
# Group the data by NODE_ID and check for duplicates in LATITUDE and LONGITUDE
duplicates_df = node_df.groupby('NODE_ID').agg({
    'LATITUDE': lambda x: len(x.unique()) > 1,
    'LONGITUDE': lambda x: len(x.unique()) > 1,
    'POSTCODE_CRASH': lambda x: len(x.unique()) > 1
}).any(axis=1)

# Print the result
print(duplicates_df[duplicates_df])

# If the result is empty, it means that NODE_ID always has the same LATITUDE and LONGITUDE
# If there are rows in the result, it means that there are instances where the same NODE_ID has different LATITUDE and LONGITUDE values


Series([], dtype: bool)


# Count num of accidents cases in a location, NODE.csv, Make num of incidents new column

In [ ]:
node_df['NUM_INCIDENTS'] = node_df.groupby('NODE_ID')['ACCIDENT_NO'].transform('count')

node_df.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,LGA NAME ALL,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS
0,T20120000009,249102,N,2563628.962,2362700.434,BAW BAW,BAW BAW,RURAL_VICTORIA,-38.2350,145.7267,3981,1
1,T20120000012,41780,N,2514194.683,2399718.586,MONASH,MONASH,MELB_URBAN,-37.9035,145.1614,3170,14
2,T20120000013,69811,I,2508689.410,2396130.003,KINGSTON,KINGSTON,MELB_URBAN,-37.9359,145.0988,3169,1
3,T20120000018,22636,I,2227360.371,2804744.001,MILDURA,MILDURA,RURAL_VICTORIA,-34.2159,142.0439,3505,2
4,T20120000021,248597,N,2480590.789,2347977.747,MORNINGTON PENINSULA,MORNINGTON PENINSULA,MELB_URBAN,-38.3696,144.7779,3942,1


# *Check accident_event

In [ ]:
accevt_df = pd.read_csv('ACCIDENT_EVENT.csv')

accevt_df.head()

,ACCIDENT_NO,EVENT_SEQ_NO,EVENT_TYPE,EVENT_TYPE_DESC,VEHICLE_1_ID,VEHICLE_1_COLL_PT,VEHICLE 1 COLL PT DESC,VEHICLE_2_ID,VEHICLE_2_COLL_PT,VEHICLE 2 COLL PT DESC,PERSON_ID,OBJECT_TYPE,OBJECT_TYPE_DESC
0,T20120003941,1,3,Ran off carriageway,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,99,Not Applicable
1,T20120003941,2,C,Collision,A,5,Left front corner,NaN,NaN,NaN,NaN,7,Guard Rail
2,T20120003941,3,C,Collision,A,9,Not known or Not Applicable,B,8,Left rear corner,NaN,99,Not Applicable
3,T20120003941,4,2,Fell from vehicle,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,99,Not Applicable
4,T20120003958,1,1,Rollover on/off carriageway,A,1,Right front corner,NaN,NaN,NaN,NaN,99,Not Applicable


In [ ]:
duplicate_entries = accevt_df['ACCIDENT_NO'].duplicated()
num_duplicate_entries = duplicate_entries.sum()
print("Number of duplicate entries:", num_duplicate_entries)

Number of duplicate entries: 110962


Comment: accident_location, accident, road_surface_cond csv files don't have duplicates, accident_event and person has duplicates

# *Check out atmospheric_cond

In [ ]:
atmo_df = pd.read_csv('ATMOSPHERIC_COND.csv')

atmo_df.head()

,ACCIDENT_NO,ATMOSPH_COND,ATMOSPH_COND_SEQ,ATMOSPH_COND_DESC
0,T20140000502,1,1,Clear
1,T20140000510,1,1,Clear
2,T20140000512,1,1,Clear
3,T20140000522,1,1,Clear
4,T20140000529,1,1,Clear


Comment: it's coorlated to individual incidents and not locations... a bit tricky

# Check out traffic light data

In [ ]:
light_df = pd.read_csv('Traffic_Lights_csv.csv')
# Rename columns to uppercase
light_df.columns = light_df.columns.str.upper()


round_lat_lon(light_df, 4)
light_df.head()

,OBJECTID,TLIGHTS_,TLIGHTS_ID,SITE_NO,SITE_NAME,SITE_TYPE,DIRECTORY,DIR_REF,D_ADDED,D_TOWNS,D_EDITED,D_REMOVED,LINK_MODE,STATUS,COMMENTS,MULTI,UFI,ARC_UFI,LATITUDE,LONGITUDE
0,45042,NaN,NaN,5168,BARWON HEADS ROAD/WARALILY BVD,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-38.2367,144.3779
1,45043,NaN,NaN,2015,MELBOURNE NR HOBART (KNOX CITY),POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-37.8692,145.2420
2,45044,NaN,NaN,6273,MITCHELL/QUEEN,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-36.7613,144.2803
3,45045,NaN,NaN,520,MOOROODUC/SAGES,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,Bluetooth,NaN,NaN,NaN,-38.1957,145.1337
4,45046,NaN,NaN,3629,CHURCHILL NR BRAYBROOK COMMUNITY CENTRE,POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-37.7861,144.8534


In [ ]:
merged_df = pd.merge(light_df, node_df, on=['LATITUDE', 'LONGITUDE'], how='inner')
num_same_coordinates = len(merged_df)
print("Number of times LATITUDE and LONGITUDE are the same in light_df and node_df:", num_same_coordinates)

Number of times LATITUDE and LONGITUDE are the same in light_df and node_df: 17137


In [ ]:
# Extract latitude, longitude, and site_no columns from light_df
site_df = light_df[['LATITUDE', 'LONGITUDE', 'SITE_NO']]
print(node_df.head())

# Merge light_data with node_df based on latitude and longitude
merged_df = pd.merge(node_df, site_df, on=['LATITUDE', 'LONGITUDE'], how='left')

# Update the site_no column in node_df with the correlated values from light_df
#node_df['SITE_NO'] = merged_df['SITE_NO']

# Display the updated node_df DataFrame
merged_df


   NODE_ID  LATITUDE  LONGITUDE  POSTCODE_CRASH  NUM_INCIDENTS
0   249102  -38.2350   145.7267            3981              1
1    41780  -37.9035   145.1614            3170             14
2    69811  -37.9359   145.0988            3169              1
3    22636  -34.2159   142.0439            3505              2
4   248597  -38.3696   144.7779            3942              1


,NODE_ID,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS,SITE_NO
0,249102,-38.2350,145.7267,3981,1,NaN
1,41780,-37.9035,145.1614,3170,14,429.0
2,69811,-37.9359,145.0988,3169,1,NaN
3,22636,-34.2159,142.0439,3505,2,NaN
4,248597,-38.3696,144.7779,3942,1,NaN
...,...,...,...,...,...,...
173755,806726,-38.1498,144.3244,3220,1,NaN
173756,1195,-38.2049,144.3406,3216,13,NaN
173757,64871,-37.7908,145.1226,3108,1,NaN
173758,807017,-37.8299,144.9641,3205,1,NaN


In [ ]:
node_df = node_df[['NODE_ID', 'LATITUDE', 'LONGITUDE', 'POSTCODE_CRASH', 'NUM_INCIDENTS']]
merged_df.to_csv('location_link.csv', index=False)

In [ ]:
num_entries = site_df['SITE_NO'].notnull().sum()
print("Number of entries where SITE_NO is not null:", num_entries)

Number of entries where SITE_NO is not null: 4852
